# Mars Seismic Data Visualization


This is a notebook to download and visualize Mars seismic event data from the xxxx 3-component seismiometer

Here is a great readme on downloading the data and formatting it from NASA: https://pds-geosciences.wustl.edu/insight/urn-nasa-pds-insight_seis/readme.txt

First need to:
 -  Choose the events by downloading the catalog (FIGURE OUT WHERE TO GET RELIABLE CATALOG #TODO)
 -  Download the raw data automatically from NASA
 -  Choose event/date time range to look at

In [13]:
import obspy
import pandas
import os
import urllib.request
import obspy
from obspy import read as obread
import matplotlib.pyplot as plt


baseurl = "https://pds-geosciences.wustl.edu/insight/urn-nasa-pds-insight_seis/data/xb/continuous_waveform/elyse/"

# Years data is avalibale 
years = [2018,2019,2020,2021,2022,2023]

# Solar days data is avalibale
soldays = [str(x).zfill(3) for x in range(367)]

#seedfile = urllib.getrequest()

testurl = "https://pds-geosciences.wustl.edu/insight/urn-nasa-pds-insight_seis/data/xb/continuous_waveform/elyse/2022/001/"
file_name = "xb.elyse.02.bhu.2022.001.8.mseed"

urllib.request.urlretrieve(testurl, file_name)
st = obread(file_name, format='MSEED')

st.plot()
plt.show()


TypeError: Format "MSEED" is not supported. Supported types: 